In [17]:
#definētas bibliotēkas un funkcijas nosaukumu tīrīšanai filmām
import pandas as pd
import requests
import numpy as np
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
import re



def preprocessing2(s):
    """
    Funkcija pārveido satura vienības nosaukumus, nometot liekos vārdus, simbolus, gadskaitļus, spec-simbolus
    """
    list_of_words=['документальный фильм','документальный','фильм [0-9]-й','фильм.[0-9]',
                   'atjaunota versija','dokumentāla animācijas filma','(angļu|krievu|vācu) val',
                   'documentaries','kara drāma','animācijas filma','animācijas īsfilma','romantiska filma',
                   'tv (pirmizrāde|pirmizrade)','(pirmizrāde|pirmizrade)','х/ф','многосерийный',
                   'spriedzes filma','komēdijdrāma','atjaunotā versija','dokumentāla drāma',
                   'dokumentāl[a-z] filmu cikls','komiska melodrāma',
                   'dokumentāls īsfilmu cikls','ltv dokumentāl[aā] filma','dokumentāla filma sērijās',
                   '(ģimenes|bērnu|pasaku|dokumentāla|piedzīvojumu|dokumentālā|fanatastikas|latviešu|mākslas|animācjas) (spēlfilma|īsfilma|filma|drāma)',
                   'atkārtojums','nosl','повтор от и все дальше','документальный проект','актёрские драмы',
                   'документальные','премьера','komedija','originalvalodas','otra dala','художественный','полная версия','комедия',
                  'актерские драмы','семейные драмы','oriģinālvalodas izvēle.','драма','фильм','rīgas kinostudijas',
                   'romantiskā filma','romantiska komēdija','biogrāfiska drāma','drama','drāma',
                  'ltv videofilma','romantiska drāma','krimināldrāma','м/ф','д/ф','(продолжение)','ar subtitriem',
                   'dokumentals','dokumentalu','komēdija','dokumentala','īsfilma','(filma|īsfilma|spēlfilma)','(new)','(dd|hd)'
                  ]
    conjoined_words = r'(?:^|[^A-Za-zЁёА-я0-9_]|\b)(?:{})(?:$|[^A-Za-zЁёА-я0-9_]|\b)'.format('|'.join(list_of_words))
    s = re.sub(conjoined_words, ' ', s)
    s = s.replace('“','')
    s = s.replace('”','')
    s = s.replace('»','')
    s = s.replace('"','')    
    s = s.replace('«','')
    s = re.sub('\([0-9]*\+\)', '', s)
    s = re.sub('–|—', '-', s)
    s = re.sub('[,\.:-]', ' ', s)
    s = re.sub('\([0-9]{4}\)', '', s)
    s = s.strip('. ')
    s = re.sub(' +', ' ', s)
    s = s.replace('^ +| +$', '')
    s = re.sub('[^a-zёА-я0-9āīūē]+$', '', s)
    return s
def preprocessing_lower(s):
    s = s.lower()
    return s

# Input

In [18]:
#epg datu ielase
input_data=pd.read_csv(r'\data_for_wikidata\movies.csv',encoding='utf8')
input_data.drop_duplicates(inplace=True)

In [3]:
input_data.head()

,title_full_original,title_full_en,title_full_lv,title_full_ru,title_full_original_pre,title_full_en_pre,title_full_lv_pre,title_full_ru_pre
0,Большой скачок. Микропроцессоры,NaN,NaN,Большой скачок. Микропроцессоры,большой скачок микропроцессоры,nav,nav,большой скачок микропроцессоры
1,"""Игорь Старыгин. Последняя дуэль"". Документальный фильм",NaN,NaN,"""Игорь Старыгин. Последняя дуэль"". Документальный фильм",игорь старыгин последняя дуэль,nav,nav,игорь старыгин последняя дуэль
2,"""Наталия Белохвостикова. Моя тайна останется со мной"". Документальный фильм",NaN,NaN,"""Наталия Белохвостикова. Моя тайна останется со мной"". Документальный фильм",наталия белохвостикова моя тайна останется со мной,nav,nav,наталия белохвостикова моя тайна останется со мной
3,В чужом краю,NaN,NaN,В чужом краю,в чужом краю,nav,nav,в чужом краю
4,La Doublure,The Valet,Dublieris,Дублер,la doublure,the valet,dublieris,дублер


# Movies

## EN filmas

In [66]:
#en nosaukumi
url = 'https://query.wikidata.org/sparql'
df_en=pd.DataFrame()

1920
662
1921
606
1922
530
1923
516
1924
544
1925
588
1926
593
1927
666
1928
662
1929
691
1930
596
1931
698
1932


JSONDecodeError: [Errno Expecting value] SPARQL-QUERY: queryStr=
    SELECT ?work ?imdb ?label_lv ?label_en ?label_ru (GROUP_CONCAT(DISTINCT ?propLabel; SEPARATOR = ",") AS ?location) (MIN(YEAR(xsd:dateTime(?when))) AS ?released) (GROUP_CONCAT(DISTINCT ?actor_; SEPARATOR = ",") AS ?cast) (GROUP_CONCAT(DISTINCT ?director_; SEPARATOR = ",") AS ?directors) WHERE {
    #   SELECT ?work ?imdb ?label_lv ?label_en ?label_ru ?propLabel ?when WHERE {
        ?work wdt:P345 ?imdb;
          wdt:P577 ?when;
          wdt:P57 ?director;
          wdt:P31 ?what;
          wdt:P161 ?actor;
          wdt:P495 ?country_of_origin;
          rdfs:label ?label_en.
        FILTER((LANG(?label_en)) = "en")
        FILTER((xsd:date(?when) >= "1932-01-01"^^xsd:date) && (xsd:date(?when) <= "1932-12-31"^^xsd:date))
        VALUES ?what {
          wd:Q11424
          wd:Q111241092
          wd:Q506240
          wd:Q24869
        }
        OPTIONAL {
          ?country_of_origin rdfs:label ?propLabel.
         

In [72]:
"""
Sparql kods datu izvilkšanai no Wikidata. Filmas, kuru nosaukums ir angļu valodā, kā arī pieejams izlaišanas gads. 
Dati iegūti iteratīvi pa gadiem
"""
years = np.arange(1939,2023,1)

for year in years:
    print(year)
    query = '''
    SELECT ?work ?imdb ?label_lv ?label_en ?label_ru (GROUP_CONCAT(DISTINCT ?propLabel; SEPARATOR = ",") AS ?location) (MIN(YEAR(xsd:dateTime(?when))) AS ?released) (GROUP_CONCAT(DISTINCT ?actor_; SEPARATOR = ",") AS ?cast) (GROUP_CONCAT(DISTINCT ?director_; SEPARATOR = ",") AS ?directors) WHERE {
    #   SELECT ?work ?imdb ?label_lv ?label_en ?label_ru ?propLabel ?when WHERE {
        ?work wdt:P345 ?imdb;
          wdt:P577 ?when;
          wdt:P57 ?director;
          wdt:P31 ?what;
          wdt:P161 ?actor;
          wdt:P495 ?country_of_origin;
          rdfs:label ?label_en.
        FILTER((LANG(?label_en)) = "en")
        FILTER((xsd:date(?when) >= "%s-01-01"^^xsd:date) && (xsd:date(?when) <= "%s-12-31"^^xsd:date))
        VALUES ?what {
          wd:Q11424
          wd:Q111241092
          wd:Q506240
          wd:Q24869
        }
        OPTIONAL {
          ?country_of_origin rdfs:label ?propLabel.
          FILTER((LANG(?propLabel)) = "en")
        }
          OPTIONAL {
          ?director rdfs:label ?director_.
          FILTER((LANG(?director_)) = "en")
        }      
          OPTIONAL {
          ?actor rdfs:label ?actor_.
          FILTER((LANG(?actor_)) = "en")
        }
        OPTIONAL {
          ?work rdfs:label ?label_lv.
          FILTER((LANG(?label_lv)) = "lv")
        }
        OPTIONAL {
          ?work rdfs:label ?label_ru.
          FILTER((LANG(?label_ru)) = "ru")
        }
      }
    # }
    GROUP BY ?work ?imdb ?label_lv ?label_en ?label_ru
    ''' %(year,year)
    r = requests.get(url, params = {'format': 'json', 'query': query})

    data = r.json(strict=False)
    print(len(data["results"]["bindings"]))
    df_en_temp=pd.json_normalize(data["results"]["bindings"])
    for column in df_en_temp.columns:
        if 'value' not in column:
            df_en_temp.drop(columns=[column],inplace=True)
    df_en=df_en.append(df_en_temp)

2011
2349
2012
2395
2013
2411
2014
2467
2015
2492
2016
2357
2017
2020
2018
1836
2019
1556
2020
1100
2021
865
2022
364


In [79]:
df_en.shape

(121196, 9)

In [101]:
#df saglabāšana
df_en.sort_values(by=['label_en.value','released.value'],ascending=True,inplace=True)
df_en.drop_duplicates(subset=['work.value', 'imdb.value', 'label_lv.value', 'label_en.value',
       'label_ru.value', 'location.value','directors.value'],inplace=True,keep='first')
print(df_en.shape)
df_en.to_csv(r'\data_for_wikidata\eng_movies_from_wiki.csv',encoding='utf8')

(111460, 9)

## RU filmas

In [181]:
#ru nosaukumi
url = 'https://query.wikidata.org/sparql'
df_ru=pd.DataFrame()

In [204]:
"""
Sparql kods datu izvilkšanai no Wikidata. Filmas, kuru nosaukums ir RU valodā, kā arī pieejams izlaišanas gads. 
Dati iegūti iteratīvi pa gadiem vai periodiem
"""
years = np.arange(1930,2030,1)
for year in years:
    print(year-0,year)
    query = '''
    SELECT ?work ?imdb ?label_lv ?label_ru (GROUP_CONCAT(DISTINCT ?propLabel; SEPARATOR = ",") AS ?location) (MIN(YEAR(xsd:dateTime(?when))) AS ?released) (GROUP_CONCAT(DISTINCT ?actor_; SEPARATOR = ",") AS ?cast) (GROUP_CONCAT(DISTINCT ?director_; SEPARATOR = ",") AS ?directors) 
    WHERE {
        ?work wdt:P345 ?imdb;
          wdt:P577 ?when;
          wdt:P57 ?director;
          wdt:P31 ?what;
          wdt:P161 ?actor;
          wdt:P495 ?country_of_origin;
          rdfs:label ?label_ru. hint:Prior hint:rangeSafe true.
            FILTER((LANG(?label_ru)) = "ru")
        FILTER((xsd:date(?when) >= "%s-01-01"^^xsd:date) && (xsd:date(?when) <= "%s-12-31"^^xsd:date))
        VALUES ?what {
          wd:Q11424
          wd:Q111241092
          wd:Q506240
          wd:Q24869
        }
        OPTIONAL {
          ?country_of_origin rdfs:label ?propLabel.
          FILTER((LANG(?propLabel)) = "en")
        }
          OPTIONAL {
          ?director rdfs:label ?director_.
          FILTER((LANG(?director_)) = "en")
        }      
          OPTIONAL {
          ?actor rdfs:label ?actor_.
          FILTER((LANG(?actor_)) = "en")
        }
          OPTIONAL {
            ?work rdfs:label ?label_en.
            FILTER((LANG(?label_en)) = "en")
          
          }
          FILTER (!bound(?label_en))
          
          OPTIONAL {
            ?work rdfs:label ?label_lv.
            FILTER((LANG(?label_lv)) = "lv")
              }
        }
    GROUP BY ?work ?imdb ?label_lv ?label_ru
    ''' %(year-0,year)
    r = requests.get(url, params = {'format': 'json', 'query': query})

    data = r.json(strict=False)
    print(len(data["results"]["bindings"]))
    df_ru_temp=pd.json_normalize(data["results"]["bindings"])
    for column in df_ru_temp.columns:
        if 'value' not in column:
            df_ru_temp.drop(columns=[column],inplace=True)
    df_ru=df_ru.append(df_ru_temp)

2019 2019
12
2020 2020
8
2021 2021
0
2022 2022
1
2023 2023
0
2024 2024
0
2025 2025
0
2026 2026


KeyboardInterrupt: 

In [205]:
#datu saglabāšana
df_ru.sort_values(by=['label_ru.value','released.value'],ascending=True,inplace=True)
df_ru.drop_duplicates(subset=['work.value', 'imdb.value', 'label_lv.value',
       'label_ru.value', 'location.value','directors.value'],inplace=True,keep='first')
print(df_ru.shape)
df_ru.to_csv(r'\data_for_wikidata\ru_movies_from_wiki.csv',encoding='utf8')

(2573, 8)


## LV filmas

In [290]:
#lv nosaukumi
url = 'https://query.wikidata.org/sparql'
df_lv=pd.DataFrame()

In [291]:
"""
Sparql kods datu izvilkšanai no Wikidata. Filmas, kuru nosaukums ir LV valodā, kā arī pieejams izlaišanas gads. 
Dati iegūti iteratīvi pa gadiem vai periodiem
"""
years = np.arange(1920,2030,1)

for year in years:
    print(year-4,year)
    query = '''
    SELECT ?work ?imdb ?label_lv (GROUP_CONCAT(DISTINCT ?propLabel; SEPARATOR = ",") AS ?location) (MIN(YEAR(xsd:dateTime(?when))) AS ?released) (GROUP_CONCAT(DISTINCT ?actor_; SEPARATOR = ",") AS ?cast) (GROUP_CONCAT(DISTINCT ?director_; SEPARATOR = ",") AS ?directors) WHERE {
      ?work wdt:P345 ?imdb;
        wdt:P577 ?when;
        wdt:P57 ?director;
        wdt:P31 ?what;
        wdt:P161 ?actor;
        wdt:P495 ?country_of_origin;
        rdfs:label ?label_lv.
      FILTER((LANG(?label_lv)) = "lv")
      FILTER(((xsd:date(?when)) >= "%s-01-01"^^xsd:date) && ((xsd:date(?when)) <= "%s-12-31"^^xsd:date))
      VALUES ?what {
        wd:Q11424
        wd:Q111241092
        wd:Q506240
        wd:Q24869
      }
      OPTIONAL {
        ?country_of_origin rdfs:label ?propLabel.
        FILTER((LANG(?propLabel)) = "en")
      }
      OPTIONAL {
        ?director rdfs:label ?director_.
        FILTER((LANG(?director_)) = "en")
      }
      OPTIONAL {
        ?actor rdfs:label ?actor_.
        FILTER((LANG(?actor_)) = "en")
      }
      OPTIONAL {
        ?work rdfs:label ?label_en.
        FILTER((LANG(?label_en)) = "en")
      }
      FILTER(!(BOUND(?label_en)))
      OPTIONAL {
        ?work rdfs:label ?label_ru.
        FILTER((LANG(?label_ru)) = "ru")
      }
      FILTER(!(BOUND(?label_ru)))
    }
    GROUP BY ?work ?imdb ?label_lv
    ''' %(year-4,year)
    r = requests.get(url, params = {'format': 'json', 'query': query})

    data = r.json(strict=False)
    print(len(data["results"]["bindings"]))
    df_lv_temp=pd.json_normalize(data["results"]["bindings"])
    for column in df_lv_temp.columns:
        if 'value' not in column:
            df_lv_temp.drop(columns=[column],inplace=True)
    df_lv=df_lv.append(df_lv_temp)

1916 1920


KeyboardInterrupt: 

In [ ]:
df_lv.sort_values(by=['label_lv.value','released.value'],ascending=True,inplace=True)
df_lv.drop_duplicates(subset=['work.value', 'imdb.value', 'label_lv.value', 'location.value','directors.value'],inplace=True,keep='first')
print(df_lv.shape)
df_lv.to_csv(r'\data_for_wikidata\lv_movies_from_wiki.csv',encoding='utf8')

## Nosaukumu tīrīšana

In [19]:
#wikidata datu apvienošana
df_1=pd.read_csv(r'\data_for_wikidata\eng_movies_from_wiki.csv',encoding='utf8',index_col=0)
df_2=pd.read_csv(r'\data_for_wikidata\ru_movies_from_wiki.csv',encoding='utf8',index_col=0)
df_3=pd.read_csv(r'\data_for_wikidata\lv_movies_from_wiki.csv',encoding='utf8',index_col=0)
df_wiki=df_1.append([df_2])
df_wiki.columns

Index(['work.value', 'imdb.value', 'label_lv.value', 'label_en.value',
       'label_ru.value', 'location.value', 'released.value', 'cast.value',
       'directors.value'],
      dtype='object')

In [21]:
# nosaukumu tīrīšana un apstrāde pirms apvienošanas - nosaukumi pārvērsti uz lowercase, 
columns_to_process=['label_lv.value', 'label_en.value', 'label_ru.value']


for column in columns_to_process:
    df_wiki[column].fillna("Nav",inplace=True)
    df_wiki[column+"_pre"]=df_wiki[column].apply(preprocessing_lower)

columns_to_process_pre=['label_lv.value_pre', 'label_en.value_pre', 'label_ru.value_pre']
for column in columns_to_process_pre:
    df_wiki[column]=np.where(df_wiki[column].str.count('"')>1,
                    df_wiki[column].str.split('"').str[1],
                    df_wiki[column])
    df_wiki[column] = df_wiki[column].apply(preprocessing2)

In [23]:
#dublikātu nomešana pēc ražošanas gada un nosaukuma.
df_wiki.sort_values(by='released.value',inplace=True, ascending=False)
(df_wiki.shape)
df_wiki.drop_duplicates(subset=['label_lv.value','label_en.value','label_ru.value'],inplace=True)

In [24]:
df_wiki.shape

(106196, 12)

## Append movies

In [27]:
#nosaukumu apvienošana pēc kādas no valodām, izmantojot original nosaukumu vai attiecīgo valodu
merged1=input_data.merge(df_wiki[df_wiki['label_en.value_pre']!='nav'], how='inner', right_on='label_en.value_pre', left_on="title_full_original_pre")
merged2=input_data.merge(df_wiki[df_wiki['label_lv.value_pre']!='nav'], how='inner', right_on='label_lv.value_pre', left_on="title_full_original_pre")
merged3=input_data.merge(df_wiki[df_wiki['label_ru.value_pre']!='nav'], how='inner', right_on='label_ru.value_pre', left_on="title_full_original_pre")
merged4=input_data.merge(df_wiki[df_wiki['label_en.value_pre']!='nav'], how='inner', right_on='label_en.value_pre', left_on="title_full_en_pre")
merged5=input_data.merge(df_wiki[df_wiki['label_lv.value_pre']!='nav'], how='inner', right_on='label_lv.value_pre', left_on="title_full_lv_pre")
merged6=input_data.merge(df_wiki[df_wiki['label_ru.value_pre']!='nav'], how='inner', right_on='label_ru.value_pre', left_on="title_full_ru_pre")

In [28]:
print(merged1.shape)
print(merged2.shape)
print(merged3.shape)
print(merged4.shape)
print(merged5.shape)
print(merged6.shape)

(4734, 20)
(530, 20)
(2772, 20)
(7495, 20)
(2132, 20)
(7472, 20)


In [29]:
#pielasīto filmu apvienošana, dublikātu nomešana EPG vienību līmenī un apvienošana ar EPG katalogu, lieko lauku nomešana
merged=pd.concat([merged1,merged2,merged3,merged4,merged5,merged6]).drop_duplicates(subset=['title_full_original', 'title_full_en', 'title_full_lv',
       'title_full_ru', 'title_full_original_pre', 'title_full_en_pre',
       'title_full_lv_pre', 'title_full_ru_pre'])
df_final=input_data.merge(merged,on=['title_full_original', 'title_full_en', 'title_full_lv',
       'title_full_ru', 'title_full_original_pre', 'title_full_en_pre',
       'title_full_lv_pre', 'title_full_ru_pre'], how='left')


df_final.drop(columns=['label_lv.value_pre','label_en.value_pre','label_ru.value_pre'], inplace=True)

In [33]:
# filmu īpatsvars, kurām pievienoti papildu dati
1-df_final['work.value'].isna().sum()/df_final.shape[0]

0.5618345048368014

In [35]:
df_final.to_csv(r'\wikidata_movies.csv', encoding='utf8', index=False)